# Building a Chatbot
## Part I: Search and Retrieval

### CORE Studio, Thornton Tomasetti

#### Instructor: [Seyedomid Sajedi](https://www.linkedin.com/in/seyedomid-sajedi-263b703a)

In this exercise, we will explore Retrieval Augmented Generation (RAG) and delve into prompt engineering for large language models. Our objective is to understand how to use these models effectively for Question-Answering (QA) tasks on custom datasets.

This notebook represents the first step in a three-part process of creating an interactive chatbot:

* Part I: Search and Retrieval
* Part II: Prompt Engineering
* Part III: Building a Chatbot UI

In Parts I and II, we will break down the key components of our custom bot. In Part III, we will integrate these components into a chatbot with a user-friendly interface. Let's get started!


---

#Libraries

In [1]:
%%capture
!pip install rank_bm25 pypdf2 tiktoken

In [2]:
# !pip install --upgrade spacy # might be needed if the default spacy in colab is not working
import spacy
import requests
from io import BytesIO
from PyPDF2 import PdfReader
from tqdm import tqdm
import tiktoken
import pickle

## spaCy
[spaCy](https://spacy.io/) is a popular and powerful natural language processing (NLP) library that excels in tasks like tokenization, part-of-speech tagging, named entity recognition. Its efficiency and pre-trained models make it a great choice for developers, researchers, and businesses looking to work with text data, enabling them to quickly and accurately analyze and extract insights from large amounts of text, with support for multiple languages and a user-friendly API.

We will use spaCy's tokenization for our exercise but it also comes with other useful features as well that we will quickly take a look at.



It is a good idea to remove english stop words from our tokenization.

In [3]:
nlp_model = spacy.load("en_core_web_sm")
# Get the list of stop words
stop_words = list(nlp_model.Defaults.stop_words)
# print(len(stop_words))
# print(stop_words)

We will use spaCy to tokenize both reference documents in our database and user search queries. The following function handles tokenization for us:

In [4]:
def spacy_tokenizer(text, nlp):
    doc = nlp(text)
    tokens = []
    for token in doc:
        # Check if the token is not punctuation and not a stop word
        if not (token.is_punct or token.is_stop):
            tokens.append(token.lemma_.lower())
    return tokens

In [5]:
openai_tokenizer = tiktoken.get_encoding("cl100k_base")

The next function will grab a pdf document from a url and loads it into memory. You can alternatively upload a document to Colab or your working directory and load it manually.

In [6]:
# We used this function for topic modeling
def get_pdf_as_memory_stream(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for HTTP errors
    # Convert the response content into a BytesIO stream
    return BytesIO(response.content)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from bs4 import BeautifulSoup

directory_path = '/content/drive/MyDrive/Learn/Coding/Revit APIs/2021.1/html/'

# Get all files in directory
files_in_directory = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))] # entire API
# files_in_directory = [f for f in os.listdir(directory_path)[0:5] if os.path.isfile(os.path.join(directory_path, f))] # subset for testing

print(len(files_in_directory))

In [ ]:
api_docs = []
for file_name in files_in_directory:
    # print(file_name)
    full_path = os.path.join(directory_path, file_name)  # Get full path to file
    with open(full_path, 'r', encoding='utf-8') as file:  # Add encoding='utf-8' to handle any special characters in the HTML files
        soup = BeautifulSoup(file, 'html.parser')
        # print(soup.title.string if soup.title else "No title")
        # print(soup)

        # syntax_section = soup.find('div', id='syntaxSection')
        # remarks_section = soup.find('div', id='remarksSection')
        # syntax_code_blocks = soup.find('div', id='syntaxCodeBlocks')
        # nsr_title = soup.find('span', id='nsrTitle')

        # output = [syntax_section, remarks_section, syntax_code_blocks, nsr_title]
        # print(output)

        csharp_code = soup.find('span', codelanguage='CSharp')

        if csharp_code:
          code = csharp_code.get_text()[2:]
        else:
          code = ""

        print(code)
        print("-"*100)

        api_docs.append(f"{code} ")
        # api_docs.append(soup)
        # api_docs.append(str(soup.prettify()))


In [ ]:
# should move condition before append to be more efficient
# don't append if item == ' '

while(' ' in api_docs):
  api_docs.remove(' ')

print(api_docs)

## Indexing the search database
Large language models have a limit cap on the amount of text they can handle for each conversation. We have to keep that in mind when building our search databse. It is ideal to chunk your long documents into small and independent sections. This smart chunking, however, could be very challenging to automate for large datasets. It is also important to keep the chunks roughly the same size. Having very long and very short chunks could adversly affect the quality of search.

For this exercise, we will simply split the document page by page. In a real project, it is often worth spending on a more delibrate strategy to index and chunk your text dataset.

In [ ]:
# tokenize dataset
tokenized_dataset= []
text_dataset =[]

# from beautiful-soup
print(tqdm)
print(len(api_docs))
for html in api_docs:
    # print(html)
    html = str(html)
    text_dataset.append(html)
    tokenized_text = spacy_tokenizer(html, nlp_model)
    print(tokenized_text)
    tokenized_dataset.append(tokenized_text)

In [ ]:
print(len(tokenized_dataset))
print(tokenized_dataset[0])

## Search Algorithm
Search algorithms play a pivotal role in various fields. Their complexity stems from the need to efficiently navigate vast datasets or solution spaces, making them indispensable tools for finding relevant information and optimizing decision-making processes in the modern world. For our search engine, we will utilize a classical and popular ranking model called [BM25](https://en.wikipedia.org/wiki/Okapi_BM25).

**Note:**

BM25 is a keyword sensitive (lexical) search algorithm. It calculates a relavance score between a given user query and reference documents. Does this sound familiar to anything we did befin the previous exercises?

In [ ]:
from rank_bm25 import BM25Okapi
def init_bm25_vectorizer():
    return BM25Okapi(tokenized_dataset)
bm25_vectorizer = init_bm25_vectorizer()

In [ ]:
# search
query="What code do i need to create a sheet in revit?"
query_tokens = spacy_tokenizer(query,nlp_model)
query_tokens

In [ ]:
bm25_scores = bm25_vectorizer.get_scores(query_tokens)
print(bm25_scores.shape)

After scoring the documents based on relevance, they can be sorted to get the top k results.

In [ ]:
import numpy as np
top_k = 3 # Number of top k pages based on BM25 score
top_page_indx= np.argsort(bm25_scores)[-top_k:][::-1]
hits = [{'page_indx': idx, 'score': bm25_scores[idx]} for idx in top_page_indx]
hits

Now, let's print the retrieved documents:

In [ ]:
import pprint
total_tokens = []
reference_list = []

for hit in hits:
  page_str = text_dataset[hit['page_indx']]
  openai_tokenizer = tiktoken.get_encoding("cl100k_base")
  token_count = len(openai_tokenizer.encode(page_str))
  total_tokens.append(token_count)
  reference_list.append(page_str)

  # print(f"PAGE: {hit['page_indx']+1}, token count:{token_count}")
  # print("\n")
  # pprint.pprint(f"PAGE: {hit['page_indx']+1}, token count:{token_count}")
  # print("\n")


  # print(page_str)
  # print("-"*50)

all_references = ('\n\n'.join(reference_list))
print(all_references)
print(sum(total_tokens))

Finally we can create a clean function for our document retrieval:

In [ ]:
# doc_retrieval('Move door at Gridline J/2 3 ft to the right', 10)
# doc_retrieval('collector = FilteredElementCollector(doc).OfCategory(BuiltInCategory.OST_StructuralColumns).WhereElementIsNotElementType()', 10)
# doc_retrieval('filtered element collector', 3)
# doc_retrieval('collector', 3)

In [ ]:
import pprint

def doc_retrieval(query,
                  top_k=3):
  #  we need to load bm25_vectorizer,nlp_model in other scripts
  query_tokens = spacy_tokenizer(query,nlp_model)
  bm25_scores = bm25_vectorizer.get_scores(query_tokens)
  top_page_indx= np.argsort(bm25_scores)[-top_k:][::-1]
  hits = [{'page_indx': idx, 'score': bm25_scores[idx]} for idx in top_page_indx]

  total_tokens = []
  reference_list = []

  print('-'*100)
  print(f"Query: {query}")
  print('-'*100)

  for hit in hits:
    page_str = text_dataset[hit['page_indx']]
    openai_tokenizer = tiktoken.get_encoding("cl100k_base")
    token_count = len(openai_tokenizer.encode(page_str))
    total_tokens.append(token_count)
    reference_list.append(page_str)
    pprint.pprint(page_str)

  print(f"\nToken count from reference text: {sum(total_tokens)}")
  print('-'*100, '\n')

  all_references = ('\n\n'.join(reference_list))
  return all_references

In [ ]:
doc_retrieval('delete all elements', 50)
